In [ ]:
pip install split-folders

In [ ]:
!apt install --allow-change-held-packages libcudnn8=8.1.0.77-1+cuda11.2

#Unzip

In [ ]:
!unzip -q tatoo1.zip

In [ ]:
import splitfolders

import os

import cv2
import numpy as np

from matplotlib import pyplot as plt
import matplotlib.cm as cm

import tensorflow as tf

import pickle

from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

from tensorflow.keras import datasets, layers, models

from tensorflow.keras.layers import Dense, Flatten, MaxPooling2D
from tensorflow.keras import Input
from tensorflow.keras.layers import Dropout, BatchNormalization

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

In [ ]:
folder_path = '/content/tatoo'
label_names = os.listdir(folder_path)
label_names

['blackwork', 'old_new_school', 'polynesian', 'ilezumi']

In [ ]:
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

dataset

In [ ]:
!mkdir resized

!mkdir resized/blackwork
!mkdir resized/ilezumi
!mkdir resized/old_new_school
!mkdir resized/polynesian

In [ ]:
dataset.items()

#Resize

In [ ]:
for label, filenames in dataset.items():
    for filename in filenames:
        img = cv2.imread(filename)

        # 이미지의 x, y가 224이 넘을 경우 작게해주기
        percent = 1
        if(img.shape[1] > img.shape[0]) :       # 이미지의 가로가 세로보다 크면 가로를 640으로 맞추고 세로를 비율에 맞춰서
            percent = 300/img.shape[1]
        else :
            percent = 300/img.shape[0]

        img = cv2.resize(img, dsize=(0, 0), fx=percent, fy=percent, interpolation=cv2.INTER_LINEAR)
                # 이미지 범위 지정
        y,x,h,w = (0,0,img.shape[0], img.shape[1])

        # 그림 주변에 검은색으로 칠하기
        w_x = (300-(w-x))/2  # w_x = (224 - 그림)을 뺀 나머지 영역 크기 [ 그림나머지/2 [그림] 그림나머지/2 ]
        h_y = (300-(h-y))/2

        if(w_x < 0):         # 크기가 -면 0으로 지정.
            w_x = 0
        elif(h_y < 0):
            h_y = 0

        M = np.float32([[1,0,w_x], [0,1,h_y]])  #(2*3 이차원 행렬)
        img_re = cv2.warpAffine(img, M, (300, 300)) #이동변환  
       
        # cv2.imwrite('{0}.jpg',image .format(file)) #파일저장
        cv2.imwrite('/content/resized/{0}/{1}'.format(label, filename.split("/")[-1]) , img_re)

In [ ]:
splitfolders.ratio('resized', output='dataset', seed=77, ratio=(0.6, 0.2, 0.2))

Copying files: 4677 files [00:00, 7905.66 files/s]


#Train Set

In [ ]:
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

label2index = {'blackwork' : 0, 'ilezumi' : 1 , 'old_new_school' : 2 , 'polynesian' : 3
               }

x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_train.append(image)
        y_train.append(label2index[label])

x_train, y_train = np.array(x_train), np.array(y_train)
x_train = x_train.astype('float32')
x_train.shape, y_train.shape

((2805, 300, 300, 3), (2805,))

#Data Augmentation

In [ ]:
datagen = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
)

In [ ]:
for label in  os.listdir(folder_path):
    label_path = folder_path + '/' + label + '/'
    for filename in os.listdir(label_path): 
        filepath = label_path + filename

        img = load_img(filepath)
        # img 출력
        # plt.imshow(img)
        # break
        x = img_to_array(img)
        # x.shape 출력
        # print(x.shape)
        # break
        x = x.reshape((1,) + x.shape)

        i = 0
        # flow : augmentation 함수. 계속 반복하므로, break 필요
        for batch in datagen.flow(x, batch_size=1,
                                save_to_dir=label_path, save_prefix=label, save_format='jpg'):
          # break
            i += 1
            if i > 3:
                break  
            

#After Data Augmentaion - Train Set

In [ ]:
folder_path = '/content/dataset/train'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

label2index = {'blackwork' : 0, 'ilezumi' : 1 , 'old_new_school' : 2 , 'polynesian' : 3
               }

x_train, y_train = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_train.append(image)
        y_train.append(label2index[label])

x_train, y_train = np.array(x_train), np.array(y_train) 

x_train = x_train.astype('float32')

x_train.shape, y_train.shape

((12546, 300, 300, 3), (12546,))

#Validation Set

In [ ]:
folder_path = '/content/dataset/val'
dataset = {}

# 이미지와 라벨 리스트에 담기
for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

x_val, y_val = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)
        x_val.append(image)
        y_val.append(label2index[label])

x_val, y_val= np.array(x_val), np.array(y_val)

x_val = x_val.astype('float32')

x_val.shape, y_val.shape

((934, 300, 300, 3), (934,))

#Test Set

In [ ]:
folder_path = '/content/dataset/test'
dataset = {}

for label in os.listdir(folder_path):
    sub_path = folder_path+'/'+label+'/'
    dataset[label] = []
    for filename in os.listdir(sub_path):
        dataset[label].append(sub_path+filename)

x_test, y_test = [], []

for label, filenames in dataset.items():
    for filename in filenames:
        image = cv2.imread(filename)

        x_test.append(image)
        y_test.append(label2index[label])

x_test, y_test = np.array(x_test), np.array(y_test)

x_test = x_test.astype('float32')

x_test.shape, y_test.shape

((938, 300, 300, 3), (938,))

#Zero Centering

In [ ]:
def zero_mean(image):
    return np.mean(image, axis=0)

zero_mean_img = zero_mean(x_train)

x_train -= zero_mean_img

x_val -= zero_mean_img
x_test -= zero_mean_img

#CNN

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu',input_shape=(300, 300, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(256, (3, 3), activation='relu'))

model.add(layers.Flatten())
model.add(layers.Dense(128, activation='relu'))
model.add(layers.Dense(32, activation='relu'))
model.add(layers.Dense(4, activation='softmax'))

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

#Model

In [ ]:
from tensorflow.keras.applications import ResNet152, ResNet50, VGG16, VGG19, AlexNet

In [ ]:
base_model = ResNet152(include_top=False, input_shape = (300,300,3), weights = 'imagenet')

base_model.trainable = True
for layer in base_model.layers[:400]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

Augmentation - 1

In [ ]:
base_model = ResNet152(include_top=False, input_shape = (300,300,3), weights = 'imagenet')

base_model.trainable = True

for layer in base_model.layers[:400]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

Augmentation - 2

In [ ]:
base_model = ResNet152(include_top=False, input_shape = (300,300,3), weights = 'imagenet')

base_model.trainable = True

for layer in base_model.layers[:400]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training=False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x= tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1)

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

1. 전체 재학습

In [ ]:
base_model = ResNet152(include_top = False, input_shape = (300, 300,3), weights = 'imagenet')

base_model.trainable = True				

# for layer in base_model.layers[:]:	
#   print(layer.name, layer.trainable)	

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training = False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(256, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])


callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.1, # new_lr = lr * factor.
    patience=2,
    min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
    min_lr=0
)

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1, callbacks = [callback])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

2. 분류기만 재학습

In [ ]:
base_model = ResNet152(include_top = False, input_shape = (300, 300,3), weights = 'imagenet')		

base_model.trainable = False


inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training = False)

x = tf.keras.layers.Flatten(input_shape = base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(2048, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(128, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.1, # new_lr = lr * factor.
    patience=2,
    min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
    min_lr=0
)

model.fit(x_train, y_train, epochs = 20, validation_data=(x_val, y_val), verbose = 1, callbacks = [callback])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

2-1 Fine_Tune(미세조정)

In [ ]:
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])

model.fit(x_train, y_train, epochs = 10, validation_data=(x_val, y_val))

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

3. 하위층 일부 재학습

In [ ]:
base_model = ResNet152(include_top = False, input_shape = (300, 300, 3), weights = 'imagenet')

for layer in base_model.layers[:-20]: 
    layer.trainable = False			

inputs = tf.keras.Input(shape=(300, 300, 3))

x = base_model(inputs, training = False)

x = tf.keras.layers.Flatten(input_shape=base_model.output_shape[1:])(x)
x = tf.keras.layers.Dense(512, activation = 'relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs, outputs)

model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics = ['accuracy'])

callback = tf.keras.callbacks.ReduceLROnPlateau(
    monitor="val_loss", 
    factor=0.1, # new_lr = lr * factor.
    patience=2,
    min_delta=0.0001, # threshold for measuring the new optimum, to only focus on significant changes.  
    min_lr=0
)                

model.fit(x_train, y_train, epochs = 15, validation_data=(x_val, y_val), verbose = 1, callbacks = [callback])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)

3-1 Fine-Tune

In [ ]:
base_model.trainable = True

model.compile(optimizer = tf.keras.optimizers.Adam( learning_rate= 0.0001),
                loss = 'sparse_categorical_crossentropy',
                metrics=['accuracy'])
early = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model.fit(x_train, y_train, epochs = 10, validation_data=(x_val, y_val), callbacks = [early])

test_loss, test_acc = model.evaluate(x_test, y_test, verbose=1)
print(test_acc)